# --- Day 10: Factory ---
https://adventofcode.com/2025/day/10

In [1]:
import re
import z3

def getInput():
	with open("schematics.txt") as file:
		return file.read()

In [2]:
lightsAndButtons = []
for line in getInput().splitlines():
	# Get the lights (the characters in the brackets)
	indicatorLightDiagram = re.findall(r'\[([.#]+)\]', line)[0]

	# Get and format buttons
	buttons = re.findall(r'\(((?:\d,?)+)\)', line)
	buttons = [list(map(int, x.split(","))) for x in buttons]
	lightsAndButtons.append((indicatorLightDiagram, buttons))


def toggleLights(currentLights: str, button: list[int]) -> str:
	"""Returns lights with all indexes in button toggled"""
	newLights = ""
	for i, light in enumerate(currentLights):
		if i in button:
			newLights += "." if light == "#" else "#"
		else:
			newLights += light
	
	return newLights

def getMinButtonPresses(goal: str, buttons: list[list[int]]) -> int:
	"""Uses bfs to get the minimum number of button presses to get to the goal"""
	queue = [["." * len(goal), 0]]
	explored = set()
	
	# Loop until the queue is empty
	while queue:
		# Get the next light layout in the queue
		# If we've hit the goal then return the number of steps it took to get there
		currentLights, steps = queue.pop(0)

		# Loop through all lights after each button press
		for successor in [toggleLights(currentLights, button) for button in buttons]:
			if successor == goal:
				return steps + 1
			if successor not in explored:
				queue.append([successor, steps + 1])
				explored.add(successor)

totalButtonPresses = 0
for lights, buttons in lightsAndButtons:
	totalButtonPresses += getMinButtonPresses(lights, buttons)

print(f"Total number of button presses: {totalButtonPresses}")

Total number of button presses: 447


# --- Part Two --- 

In [3]:
joltageAndButtons = []

for line in getInput().splitlines():
	# Get the joltage requirements as a list
	joltageRequirements = re.findall(r'(?:{((?:\d+,?)+)})', line)[0]
	joltageRequirements = list(map(int, joltageRequirements.split(",")))

	# Get and format buttons
	buttons = re.findall(r'\(((?:\d,?)+)\)', line)
	buttons = [list(map(int, x.split(","))) for x in buttons]
	
	joltageAndButtons.append((joltageRequirements, buttons))

def getMinButtonPresses(joltages: list[int], buttons: list[list[int]]) -> int:
	"""Uses Z3 to find minimum number of button presses to match joltage requirements"""
	# Get all button variables
	buttonVariables = []
	for character in range(ord("A"), ord("A") + len(buttons)):
		buttonVariables.append(z3.Int("Button" + chr(character)))

	# Create a solver and start adding constraints
	solver = z3.Optimize()
	for variable in buttonVariables:
		solver.add(variable >= 0)

	# Add all sum constraints (e.g. the sum of buttonA and buttonB have to equal whatever joltage they share)
	for joltageIndex, joltage in enumerate(joltages):
		# Get all button variables whose button affects the current joltage
		currentActiveButtons = [buttonVariables[i] for i, button in enumerate(buttons) if joltageIndex in button]
		solver.add(z3.Sum(currentActiveButtons) == joltage)

	# Define a cost (the number of button presses)
	buttonPresses = z3.Int("cost")
	solver.add(buttonPresses == z3.Sum(buttonVariables))

	# Return the minimized cost (fewest number of button presses)
	minCost = solver.minimize(buttonPresses)
	solver.check()
	return solver.lower(minCost).as_long()

totalButtonPresses = 0
for joltage, buttons in joltageAndButtons:
	totalButtonPresses += getMinButtonPresses(joltage, buttons)

print(f"Total number of button presses: {totalButtonPresses}")

Total number of button presses: 18960
